In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt  
import pandas as pd
from zipfile import ZipFile
from tensorflow import keras 
from kaggle.api.kaggle_api_extended import KaggleApi
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from PIL import Image
from tensorflow.keras import layers, models, optimizers 
import imageio
from tqdm import tqdm

In [2]:
dataset_path = "./celeba"

os.makedirs(dataset_path, exist_ok=True)

api = KaggleApi()
api.authenticate()
print("Downloading dataset...")
api.dataset_download_files('zuozhaorui/celeba', path=dataset_path, unzip=True)
print("Dataset downloaded and extracted to:", dataset_path)

Dataset URL: https://www.kaggle.com/datasets/zuozhaorui/celeba


KeyboardInterrupt: 

In [ ]:
IMG_SIZE = 64
BATCH_SIZE = 64

In [ ]:
DATA_DIR = "/content/img_align_celeba/img_align_celeba"
SAVE_DIR = "/content/dcgan_faces"
ddatagen = ImageDataGenerator(
    rescale=1./127.5,      
    preprocessing_function=lambda x: x - 1.0   
)

train_generator = datagen.flow_from_directory(
    directory=os.path.dirname(DATA_DIR),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode=None,
    shuffle=True
)

In [ ]:
def build_generator():
    model = models.Sequential()
    model.add(layers.Dense(8*8*512, input_dim=100))
    model.add(layers.Reshape((8, 8, 512)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(0.2))

    model.add(layers.Conv2DTranspose(256, 4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(0.2))

    model.add(layers.Conv2DTranspose(128, 4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(0.2))

    model.add(layers.Conv2DTranspose(3, 4, strides=2, padding='same', activation='tanh'))
    return model

In [ ]:
def build_discriminator():
    model = models.Sequential()
    model.add(layers.Conv2D(64, 4, strides=2, padding='same', input_shape=(64, 64, 3)))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, 4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, 4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [ ]:
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
                      optimizer=optimizers.Adam(0.0002, 0.5),
                      metrics=['accuracy'])

In [ ]:
generator = build_generator()

z = layers.Input(shape=(100,))
img = generator(z)
discriminator.trainable = False
valid = discriminator(img)

combined = models.Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(0.0002, 0.5))

In [ ]:
EPOCHS = 5000
SAVE_INTERVAL = 500
fixed_noise = np.random.normal(0, 1, (25, 100))

steps_per_epoch = len(train_generator)

for epoch in range(EPOCHS):
    # --- Load a batch of real images ---
    real_imgs = next(train_generator)
    real_imgs = real_imgs[:BATCH_SIZE]   

    # --- Generate fake images ---
    noise = np.random.normal(0, 1, (BATCH_SIZE, 100))
    fake_imgs = generator.predict(noise)

    # --- Train Discriminator ---
    d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((BATCH_SIZE, 1)))
    d_loss_fake = discriminator.train_on_batch(fake_imgs, np.zeros((BATCH_SIZE, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # --- Train Generator ---
    noise = np.random.normal(0, 1, (BATCH_SIZE, 100))
    g_loss = combined.train_on_batch(noise, np.ones((BATCH_SIZE, 1)))

    # --- Print progress ---
    if epoch % 100 == 0:
        print(f"{epoch} [D loss: {d_loss[0]:.4f}, acc: {100*d_loss[1]:.2f}%] [G loss: {g_loss:.4f}]")

    # --- Save sample images ---
    if epoch % SAVE_INTERVAL == 0:
        gen_imgs = generator.predict(fixed_noise)
        gen_imgs = 0.5 * gen_imgs + 0.5
        fig, axs = plt.subplots(5, 5, figsize=(5,5))
        cnt = 0
        for i in range(5):
            for j in range(5):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(f"{SAVE_DIR}/face_{epoch}.png")
        plt.close()

In [ ]:
import glob
images_gif = []
for file in sorted(glob.glob(f"{SAVE_DIR}/face_*.png"), key=os.path.getmtime):
    images_gif.append(imageio.imread(file))
imageio.mimsave(f"{SAVE_DIR}/progress.gif", images_gif, duration=0.5)
print("✅ GIF created:", f"{SAVE_DIR}/progress.gif")